In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, simplify, expand, lambdify, Float, pretty

Matplotlib is building the font cache; this may take a moment.


In [2]:
def diferencas_finitas(y_vals):
    """Calcula a tabela de diferenças finitas progressivas."""
    n = len(y_vals)
    diff_table = [y_vals.copy()]
    for i in range(1, n):
        nova_linha = [diff_table[i-1][j+1] - diff_table[i-1][j] for j in range(n - i)]
        diff_table.append(nova_linha)
    return diff_table

In [3]:
def interpolacao_gregory_newton(x_vals, y_vals):
    """
    Aplica o método de interpolação de Gregory-Newton progressivo.

    Parâmetros:
    x_vals -- Lista de coordenadas x dos pontos (deve ser uniformemente espaçada)
    y_vals -- Lista de coordenadas y dos pontos

    Retorna:
    Polinômio interpolador e função lambdificada
    """
    x = symbols('x')
    h = x_vals[1] - x_vals[0]
    t = (x - x_vals[0]) / h

    diff_table = diferencas_finitas(y_vals)
    n = len(x_vals)
    polinomio = 0
    termo_t = 1

    for i in range(n):
        polinomio += diff_table[i][0] * termo_t / np.math.factorial(i)
        termo_t *= (t - i)

    polinomio = simplify(expand(polinomio))
    coeffs = polinomio.as_poly(x).all_coeffs()
    arredondados = [Float(c.evalf(), 4) for c in coeffs]
    polinomio = sum(c * x**i for i, c in enumerate(reversed(arredondados)))

    f_interp = lambdify(x, polinomio, 'numpy')
    return polinomio, f_interp

In [ ]:
# Interpolacao de Gregory-Newton
print("Interpolacao de Gregory-Newton")
n = int(input("Digite o numero de pontos (n): "))
x_vals = []
y_vals = []

for i in range(n):
    x = float(input(f"x{i}: "))
    y = float(input(f"y{i}: "))
    x_vals.append(x)
    y_vals.append(y)

if not np.allclose(np.diff(x_vals), x_vals[1] - x_vals[0]):
    raise ValueError("Os valores de x devem ser igualmente espaçados para Gregory-Newton.")

P, f_interp = interpolacao_gregory_newton(x_vals, y_vals)

print("\nPolinomio interpolador de Gregory-Newton:")
print(pretty(P, use_unicode=True))

# Avaliacao e grafico
x_plot = np.linspace(min(x_vals)-1, max(x_vals)+1, 500)
y_plot = f_interp(x_plot)

plt.plot(x_plot, y_plot, label='Polinomio de Gregory-Newton')
plt.scatter(x_vals, y_vals, color='red', label='Pontos dados')
plt.title('Interpolacao de Gregory-Newton')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()